In [1]:
import numpy as np
import pandas as pd
import gc
import seaborn as sns
import xgboost as xgb
import lightgbm as lgb
import math
import lightgbm as lgb
from collections import Counter  
import time
from scipy.stats import kurtosis,iqr
import seaborn as sns
from scipy import ptp
from tqdm import tqdm
from datetime import timedelta
import datetime
from sklearn.metrics import accuracy_score, roc_auc_score,log_loss,f1_score
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
import warnings
import os
from sklearn.utils import shuffle
from sklearn.externals import joblib
from mpl_toolkits.mplot3d import Axes3D
pd.options.display.max_columns = None
pd.options.display.max_rows = None
import warnings
warnings.filterwarnings('ignore')

## 处理short_stay数据
## 将short_stay和area_passenger_index合并

In [2]:
### 北京市网格人流量指数,200*200网格
cols = ['date', 'hour', 'Grid_x_1', 'Grid_y_1', 'index_1']
short_stay = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/shortstay_20200201_20200215.csv', names=cols, encoding='gb2312', sep='\t')
tem = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/shortstay_20200117_20200131.csv', names=cols, encoding='gb2312', sep='\t')
short_stay = pd.concat([short_stay, tem], axis=0)
short_stay['date'] = short_stay['date'].apply(lambda x: ''.join(str(x)[0:4]+'-'+str(x)[4:6]+'-'+str(x)[6:8]))

In [13]:
### 读取区域人流量指数，含标签
area_passenger_index_chunk = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/area_passenger_index.csv', chunksize=1000)

In [8]:
### 找对area_passenger_index中重点区域中心最近的坐标，并将其index添加
### 先将area_passenger_index和short_stay merge
area_passenger_index = pd.DataFrame()
for chunk in tqdm(area_passenger_index_chunk):
    chunk = chunk.merge(short_stay, how='left', on=['date', 'hour'])
    chunk['distance'] = (chunk['Grid_x']-chunk['Grid_x_1'])**2 + (chunk['Grid_y']-chunk['Grid_y_1'])**2
    chunk['month'] = chunk['date'].apply(lambda x: int(str(x)[5:7]))
    chunk['day'] = chunk['date'].apply(lambda x: int(str(x)[8:]))
    chunk  = chunk.drop(['date'], axis=1)
    ### 按照month,day,hour,distance排序
    chunk= chunk.sort_values(by=['month', 'day', 'hour', 'distance'], ascending=[True, True, True, True])
    ### 按照month,day,hour多列去重复，取第一次出现的数据，实现最小距离的获取
    chunk = chunk.drop_duplicates(subset=['month','day', 'hour'],keep='first')
    ### 处理一个chunk之后合并
    area_passenger_index = pd.concat([area_passenger_index, chunk], axis=0)

1it [05:01, 301.52s/it]ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/poac/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3291, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-78c61c461dfc>", line 5, in <module>
    chunk = chunk.merge(short_stay, how='left', on=['date', 'hour'])
  File "/home/poac/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py", line 6868, in merge
    copy=copy, indicator=indicator, validate=validate)
  File "/home/poac/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 48, in merge
    return op.get_result()
  File "/home/poac/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py", line 560, in get_result
    concat_axis=0, copy=self.copy)
  File "/home/poac/anaconda3/lib/python3.6/site-packages/pandas/core/internals/managers.py", line 2061, in concatenate_block_managers
    concatenate_join_units(join_units, concat_axis, copy=copy),
  File "/home/poac/anaconda3/lib/

KeyboardInterrupt: 

In [12]:
### 处理完之后再去一次重
area_passenger_index = area_passenger_index.drop_duplicates(subset=['month','day', 'hour'],keep='first')

(720, 17)

In [ ]:
### 将处理完的数据保存
area_passenger_index.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/short_stay_processed.csv', index=None)
del area_passenger_index
del area_passenger_index_chunk

## 将short_stay和input_for_sub合并

In [3]:
input_for_sub_chunk = pd.read_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub.csv', chunksize=1000)

In [4]:
### input_for_sub，并将其index添加
### 先将input_for_sub和short_stay merge
input_for_sub = pd.DataFrame()
for chunk in tqdm(input_for_sub_chunk):
    chunk = chunk.merge(short_stay, how='left', on=['date', 'hour'])
    chunk['distance'] = (chunk['Grid_x']-chunk['Grid_x_1'])**2 + (chunk['Grid_y']-chunk['Grid_y_1'])**2
    chunk['month'] = chunk['date'].apply(lambda x: int(str(x)[5:7]))
    chunk['day'] = chunk['date'].apply(lambda x: int(str(x)[8:]))
    chunk  = chunk.drop(['date'], axis=1)
    ### 按照month,day,hour,distance排序
    chunk= chunk.sort_values(by=['month', 'day', 'hour', 'distance'], ascending=[True, True, True, True])
    ### 按照month,day,hour多列去重复，取第一次出现的数据，实现最小距离的获取
    chunk = chunk.drop_duplicates(subset=['month','day', 'hour'],keep='first')
    ### 处理一个chunk之后合并
    input_for_sub = pd.concat([input_for_sub, chunk], axis=0)
input_for_sub = input_for_sub.drop_duplicates(subset=['month','day', 'hour'],keep='first')

6it [27:42, 276.46s/it]

KeyboardInterrupt: 

In [ ]:
### 保存
input_for_sub.to_csv('/home/poac/AnomalyDetectionDataset/Block_predict/processed/input_for_sub_processed.csv', index=None)